# Pygame

Pygame is a set of Python modules designed for writing video games. Pygame adds functionality on top of the excellent [SDL](https://www.libsdl.org/) library. This allows you to create fully featured games and multimedia programs in the python language.

Pygame is highly portable and runs on nearly every platform and operating system.

## The basics

In this notebook you'll find a step by step instructions to learn, test and run simple platform-like game. 

The simplest pygame program will look like the following code: it import the module, initialize the framework and execute the game's main loop that will handle the user events.


In [2]:
import pygame as pg
from os import path


pg.init()

pg.display.set_caption("First Game")


class Config:
    screen_height = 500
    screen_width = 500
    
    
def run_game():
    run = True
    screen = pg.display.set_mode((Config.screen_width, Config.screen_height))
    while run:
        for event in pg.event.get():
            if event.type == pg.QUIT:
                run = False
    pg.quit()
    
run_game()

pygame 2.2.0 (SDL 2.0.22, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


Notice the `Config` class; it will contains all the configurations needed.

Lets explore how cordinates works by adding a rectangle to the screen. Pygame uses a 2D coordinate system where the origin: (0, 0) is the top left corner of the canvas. 

The following funcion draw a rectangle in the screen and update it.

In [3]:
class Config:
    screen_height = 500
    screen_width = 500
    width_of_rec = 40
    height_of_rec = 60
    position_on_x = 50
    position_on_y = 400
    red = (255, 0, 0)
    black = (0, 0, 0)


def draw_rectangle(screen):
    pg.draw.rect(screen, Config.red, (Config.position_on_x, Config.position_on_y, Config.width_of_rec, Config.height_of_rec))
    pg.display.update()
    

def run_game():
    run = True
    screen = pg.display.set_mode((Config.screen_width, Config.screen_height))
    while run:
        for event in pg.event.get():
            if event.type == pg.QUIT:
                run = False
        draw_rectangle(screen)
    pg.quit()
    
run_game()

Lets make the rectangle move left and right; the following funciton recive the key that was pressed and change the rectangle position accordingly

In [5]:
class Config:
    screen_height = 500
    screen_width = 500
    width_of_rec = 40
    height_of_rec = 60
    position_on_x = 50
    position_on_y = 400
    red = (255, 0, 0)
    black = (0, 0, 0)
    # amount of pixels to move
    speed = 5

    
def check_move(keys):
    if keys[pg.K_LEFT] and Config.position_on_x > Config.speed:
        Config.position_on_x -= Config.speed

    if keys[pg.K_RIGHT] and Config.position_on_x < Config.screen_width - Config.width_of_rec:
        Config.position_on_x += Config.speed
        
    if keys[pg.K_UP] and Config.position_on_x > Config.speed:
        Config.position_on_y -= Config.speed

    if keys[pg.K_DOWN] and Config.position_on_y < Config.screen_height - Config.width_of_rec:
        Config.position_on_y += Config.speed    

def run_game():
    run = True
    screen = pg.display.set_mode((Config.screen_width, Config.screen_height))
    while run:
        for event in pg.event.get():
            if event.type == pg.QUIT:
                run = False
        
        keys = pg.key.get_pressed() # return the key pressed by user
        check_move(keys)
        draw_rectangle(screen)
        
        
    pg.quit()
    
run_game()

The movement is too fast to be usable for a game. The event loop iterates too fast, so we need some time for it to be "playable". Let's add some delay of 100 ms.

In [7]:
def run_game():
    run = True
    screen = pg.display.set_mode((Config.screen_width, Config.screen_height))
    while run:
        pg.time.delay(100)              # NEW
        for event in pg.event.get():
            if event.type == pg.QUIT:
                run = False
        
        keys = pg.key.get_pressed()
        check_move(keys)
        draw_rectangle(screen)
        
        
    pg.quit()
    
run_game()

As you can notice the rectangle is leaving a trace. Lets fix that by filling the screen after every move:

In [8]:
def run_game():
    run = True
    screen = pg.display.set_mode((Config.screen_width, Config.screen_height))
    while run:
        pg.time.delay(100)
        for event in pg.event.get():
            if event.type == pg.QUIT:
                run = False
        
        keys = pg.key.get_pressed() 
        check_move(keys)
        draw_rectangle(screen)
        
        screen.fill(Config.black)        # NEW

        
    pg.quit()
    
run_game()

What if the rectagle can jump instead of just navigate in the screen for no reason? Let's also make it jump with some physics law in mind, so acceleration is considered:

In [9]:
class Config:
    screen_height = 500
    screen_width = 500
    width_of_rec = 40
    height_of_rec = 60
    position_on_x = 50
    position_on_y = 400
    speed = 5
    black = (0, 0, 0)
    red = (255, 0, 0)
    jump_count = 10
    is_jump = True                        # NEW
    

def jump(keys):                           # NEW
    if Config.is_jump:
        if keys[pg.K_SPACE]:
            Config.is_jump = False
    else:
        if Config.jump_count >= -10:
            neg = 1
            if Config.jump_count < 0:
                neg = -1
            Config.position_on_y -= (Config.jump_count ** 2) / 2 * neg
            Config.jump_count -= 1
        else:
            Config.is_jump = True
            Config.jump_count = 10


def run_game():
    run = True
    screen = pg.display.set_mode((Config.screen_width, Config.screen_height))
    while run:
        pg.time.delay(100)
        for event in pg.event.get():
            if event.type == pg.QUIT:
                run = False
        
        keys = pg.key.get_pressed()
        jump(keys)                        # NEW
        check_move(keys)
        draw_rectangle(screen)
        
        screen.fill(Config.black)

        
    pg.quit()
    
run_game()

Now that the square jumps, there is no need for it to go up and down, since being a platform game it wouldn't make much sense. So let's refactor the code for the `check_move()` function.

In [11]:
def check_move(keys):                     # MODIFIED
    if keys[pg.K_LEFT] and Config.position_on_x > Config.speed: 
        Config.position_on_x -= Config.speed

    if keys[pg.K_RIGHT] and Config.position_on_x < Config.screen_width - Config.width_of_rec:
        Config.position_on_x += Config.speed

def run_game():
    run = True
    screen = pg.display.set_mode((Config.screen_width, Config.screen_height))
    while run:
        pg.time.delay(100)
        for event in pg.event.get():
            if event.type == pg.QUIT:
                run = False
        
        keys = pg.key.get_pressed()
        jump(keys)                       
        check_move(keys)
        draw_rectangle(screen)
        
        screen.fill(Config.black)

        
    pg.quit()
    
run_game()

A red square was a simple way to introduce the event loop to handle keyboard events, but let's get more realistic. To make it look more like a game let's add an actual character and animate it. Some additional variables are added to help knowing the position and movement of the character. 
Since there are so many images, loading them in lists may be useful later. The animations are made of different images of the character in different key frames.

Let's also add a background image.

In [12]:
class Config:
    screen_height = 500
    screen_width = 500
    width_of_rec = 40
    height_of_rec = 60
    position_on_x = 50
    position_on_y = 400
    speed = 5
    black = (0, 0, 0)
    red = (255, 0, 0)
    jump_count = 10
    is_jump = True
    # NEW
    walk_right = [pg.image.load(path.join("gamespy","Game","Right", f'R{n}.png')) for n in range(1, 10)]
    walk_left = [pg.image.load(path.join("gamespy","Game","Left", f'L{n}.png')) for n in range(1, 10)]
    char = pg.image.load(path.join("gamespy","Game",'standing.png'))
    bg = pg.image.load(path.join("gamespy", "Game", "bg.jpg"))
    right = False
    left = False
    walk_count = 0


The following function takes care of animating the character and setting the background. The idea is the animations will last for 3 frames.

In [13]:
def draw_game(screen):
    screen.blit(Config.bg, (0, 0))
    if Config.walk_count + 1 >= 27:
        Config.walk_count = 0

    if Config.left:
        screen.blit(Config.walk_left[Config.walk_count // 3], (Config.position_on_x, Config.position_on_y))
        Config.walk_count += 1
    elif Config.right:
        screen.blit(Config.walk_right[Config.walk_count // 3], (Config.position_on_x, Config.position_on_y))
        Config.walk_count += 1
    else:
        screen.blit(Config.char, (Config.position_on_x, Config.position_on_y))
    pg.display.update()

Change the `check_move()` function in order to know the direction the character is moving:

In [16]:
def check_move(keys):
    if keys[pg.K_LEFT] and Config.position_on_x > Config.speed:
        Config.position_on_x -= Config.speed
        Config.left = True
        Config.right = False

    elif keys[pg.K_RIGHT] and Config.position_on_x < Config.screen_width - Config.width_of_rec:
        Config.position_on_x += Config.speed
        Config.right = True
        Config.left = False
    else:
        Config.right = False
        Config.left = False
        Config.walk_count = 0
        

def run_game():
    run = True
    screen = pg.display.set_mode((Config.screen_width, Config.screen_height))
    while run:
        pg.time.delay(100)
        for event in pg.event.get():
            if event.type == pg.QUIT:
                run = False

        keys = pg.key.get_pressed()
        jump(keys)
        check_move(keys)
        draw_game(screen)         # NEW
    pg.quit()


run_game()

The animations are not as smooth. This is due the delay introduced before. That was fine for the square but pygame offers a better alternative than a fix delay. Let's add a `clock` that tick so we get 27 FPS, a bit higher than a movie so the game looks smoother.

In [17]:
class Config:
    screen_height = 500
    screen_width = 500
    width_of_rec = 40
    height_of_rec = 60
    position_on_x = 50
    position_on_y = 400
    speed = 5
    black = (0, 0, 0)
    red = (255, 0, 0)
    jump_count = 10
    is_jump = True
    walk_right = [pg.image.load(path.join("gamespy","Game","Right", f'R{n}.png')) for n in range(1, 10)]
    walk_left = [pg.image.load(path.join("gamespy","Game","Left", f'L{n}.png')) for n in range(1, 10)]
    char = pg.image.load(path.join("gamespy","Game",'standing.png'))
    bg = pg.image.load(path.join("gamespy", "Game", "bg.jpg"))
    right = False
    left = False
    walk_count = 0
    # NEW
    clock = pg.time.Clock()
    

def run_game():
    run = True
    screen = pg.display.set_mode((Config.screen_width, Config.screen_height))
    while run:
        
        Config.clock.tick(27) # the game will be at 27 FPS 
        
        for event in pg.event.get():
            if event.type == pg.QUIT:
                run = False

        keys = pg.key.get_pressed()
        jump(keys)
        check_move(keys)
        draw_game(screen)
    pg.quit()


run_game()

Let's refactor the code a bit by separating the character configuration from the game configurations. Since our character will no longer be a rectangle let's change the name of the variables `width_of_rec` and `height_of_rec `

In [19]:
class Player:
    position_on_x = 300
    position_on_y = 410
    width = 64
    height = 64
    right = False
    left = False
    walk_count = 0
    jump_count = 10
    is_jump = True
    speed = 5
    walk_right = [pg.image.load(path.join("gamespy","Game","Right", f'R{n}.png')) for n in range(1, 10)]
    walk_left = [pg.image.load(path.join("gamespy","Game","Left", f'L{n}.png')) for n in range(1, 10)]
    char = pg.image.load(path.join("gamespy","Game",'standing.png'))


class Config:
    screen_height = 500
    screen_width = 500
    black = (0, 0, 0)
    red = (255, 0, 0)
    clock = pg.time.Clock()
    bg = pg.image.load(path.join("gamespy", "Game", "bg.jpg"))

Change `Config` to `Player` in their respective places

In [21]:

def draw_game(screen):
    screen.blit(Config.bg, (0, 0))
    if Player.walk_count + 1 >= 27:
        Player.walk_count = 0

    if Player.left:
        screen.blit(Player.walk_left[Player.walk_count // 3], (Player.position_on_x, Player.position_on_y))
        Player.walk_count += 1
    elif Player.right:
        screen.blit(Player.walk_right[Player.walk_count // 3], (Player.position_on_x, Player.position_on_y))
        Player.walk_count += 1
    else:
        screen.blit(Player.char, (Player.position_on_x, Player.position_on_y))
    pg.display.update()


def check_move(keys):
    if keys[pg.K_LEFT] and Player.position_on_x > Player.speed:
        Player.position_on_x -= Player.speed
        Player.left = True
        Player.right = False

    elif keys[pg.K_RIGHT] and Player.position_on_x < Config.screen_width - Player.width:
        Player.position_on_x += Player.speed
        Player.right = True
        Player.left = False
    else:
        Player.right = False
        Player.left = False
        Player.walk_count = 0


def jump(keys):
    if Player.is_jump:

        if keys[pg.K_SPACE]:
            Player.is_jump = False
            Player.right = False
            Player.left = False
            Player.walk_count = 0

    else:
        if Player.jump_count >= -10:
            neg = 1
            if Player.jump_count < 0:
                neg = -1
            Player.position_on_y -= (Player.jump_count ** 2) / 2 * neg
            Player.jump_count -= 1
        else:
            Player.is_jump = True
            Player.jump_count = 10


def run_game():
    run = True
    screen = pg.display.set_mode((Config.screen_width, Config.screen_height))
    while run:
        Config.clock.tick(27)
        for event in pg.event.get():
            if event.type == pg.QUIT:
                run = False

        keys = pg.key.get_pressed()
        jump(keys)
        check_move(keys)
        draw_game(screen)
    pg.quit()


run_game()

Let's make the character to throw some projectiles, to do that some changes need to be introduced, the character can't stay looking "to the screen" while is standing still because the projectile could not go out to any of the sides. Let's add the variable `standing` to `Player`, to identify when it is still. Then change the `draw_game()` function and the `check_move()` function as shown:

In [22]:
class Player:
    position_on_x = 300
    position_on_y = 410
    width = 64
    height = 64
    right = False
    left = False
    walk_count = 0
    jump_count = 10
    is_jump = True
    speed = 5
    walk_right = [pg.image.load(path.join("gamespy","Game","Right", f'R{n}.png')) for n in range(1, 10)]
    walk_left = [pg.image.load(path.join("gamespy","Game","Left", f'L{n}.png')) for n in range(1, 10)]
    char = pg.image.load(path.join("gamespy","Game",'standing.png'))

    standing = True                      # NEW


def draw_game(screen):
    screen.blit(Config.bg, (0, 0))
    if Player.walk_count + 1 >= 27:
        Player.walk_count = 0

    if Player.left:
        screen.blit(Player.walk_left[Player.walk_count // 3], (Player.position_on_x, Player.position_on_y))
        Player.walk_count += 1
    elif Player.right:
        screen.blit(Player.walk_right[Player.walk_count // 3], (Player.position_on_x, Player.position_on_y))
        Player.walk_count += 1
    else:
        if Player.right:                # NEW
            screen.blit(Player.walk_right[0], (Player.position_on_x, Player.position_on_y))
        else:
            screen.blit(Player.walk_left[0], (Player.position_on_x, Player.position_on_y))
            
            
    pg.display.update()
    
    
    
def check_move(keys):
    if keys[pg.K_LEFT] and Player.position_on_x > Player.speed:
        Player.position_on_x -= Player.speed
        Player.left = True
        Player.right = False
        Player.standing = False           # NEW

    elif keys[pg.K_RIGHT] and Player.position_on_x < Config.screen_width - Player.width:
        Player.position_on_x += Player.speed
        Player.right = True
        Player.left = False
        Player.standing = False          # NEW

    else:
        Player.standing = False          # NEW
        Player.walk_count = 0
        
        

def run_game():
    run = True
    screen = pg.display.set_mode((Config.screen_width, Config.screen_height))
    while run:
        Config.clock.tick(27)
        for event in pg.event.get():
            if event.type == pg.QUIT:
                run = False

        keys = pg.key.get_pressed()
        jump(keys)
        check_move(keys)
        draw_game(screen)
    pg.quit()


run_game()

Now let's create the `Projectile` class:

In [23]:
class Projectile(object):
    def __init__(self, position_on_x, position_on_y, radius, color, facing):
        self.position_on_x = position_on_x
        self.position_on_y = position_on_y
        self.radius = radius
        self.color = color
        self.facing = facing
        self.speed = 8 * facing

    def draw(self, screen):
        pg.draw.circle(screen, self.color, (self.position_on_x, self.position_on_y), self.radius)

Let's introduce another refactor by extracting from the `draw_game()` function the logic that's relevant for the player into a `draw_playe()` function, let's do this so the new logic for the proyectiles won't create much clutter:

In [24]:
def draw_player(screen):
    if Player.walk_count + 1 >= 27:
        Player.walk_count = 0

    if Player.left:
        screen.blit(Player.walk_left[Player.walk_count // 3], (Player.position_on_x, Player.position_on_y))
        Player.walk_count += 1
    elif Player.right:
        screen.blit(Player.walk_right[Player.walk_count // 3], (Player.position_on_x, Player.position_on_y))
        Player.walk_count += 1
    else:
        if Player.right:
            screen.blit(Player.walk_right[0], (Player.position_on_x, Player.position_on_y))
        else:
            screen.blit(Player.walk_left[0], (Player.position_on_x, Player.position_on_y))


def draw_game(screen, bullets):
    screen.blit(Config.bg, (0, 0))
    draw_player(screen)                  # NEW
    pg.display.update()

Bullets will be created using lists, with a maximum of 3 of them, making sure that when they go beyond the limits of the screen they will be deleted with the `pop` method. After this the list will be reloaded with each bullet popped. ALso instruct `draw_game()` to draw the bullets list. Change the controls to shoot with the space bar and jump with the arrow key.

In [26]:
def draw_game(screen, bullets):
    screen.blit(Config.bg, (0, 0))
    
    for bullet in bullets: # NEW  
        bullet.draw(screen)
        
    draw_player(screen)
    pg.display.update()

    
def jump(keys):
    if Player.is_jump:
        if keys[pg.K_UP]:                    # NEW: arrow key to jump
            Player.is_jump = False
            Player.right = False
            Player.left = False
            Player.walk_count = 0

    else:
        if Player.jump_count >= -10:
            neg = 1
            if Player.jump_count < 0:
                neg = -1
            Player.position_on_y -= (Player.jump_count ** 2) / 2 * neg
            Player.jump_count -= 1
        else:

            Player.is_jump = True
            Player.jump_count = 10    
    
    
    
def run_game():
    bullets = []
    run = True
    screen = pg.display.set_mode((Config.screen_width, Config.screen_height))
    while run:

        Config.clock.tick(27)

        for event in pg.event.get():
            if event.type == pg.QUIT:
                run = False
     
        for bullet in bullets:                # NEW
            if 500 > bullet.position_on_x > 0:
                bullet.position_on_x += bullet.speed
            else:
                bullets.pop(bullets.index(bullet))
        keys = pg.key.get_pressed()
        if keys[pg.K_SPACE]:                  # shoot with space bar
            if Player.left:
                facing = -1
            else:
                facing = 1
    
            if len(bullets) < 3:              # adding bullets to the list with Projectile properties
                bullets.append(
                    Projectile(round(Player.position_on_x + Player.width // 2),
                               round(Player.position_on_y + Player.height // 2), 6, Config.black,
                               facing))
                
        jump(keys)
        check_move(keys)
        draw_game(screen, bullets)
    pg.quit()


run_game()

Create the Enemy class, where the images will be loaded in the same way we did before to make the animations, also add the other properties and methods to the class.

In [29]:
class Enemy(object):
    walk_right = [pg.image.load(path.join("gamespy","Game","RightE", f'R{n}E.png')) for n in range(1, 12)]
    walk_left = [pg.image.load(path.join("gamespy","Game","LeftE", f'L{n}E.png')) for n in range(1, 12)]
    
    def __init__(self, position_on_x, position_on_y, width, height, end):
        self.position_on_x = position_on_x
        self.position_on_y = position_on_y
        self.width = width
        self.height = height
        self.path = [position_on_x, end]  # This will define where our enemy starts and finishes their path.
        self.walk_count = 0
        self.speed = 3

    def draw(self, screen):
        self.move()
        if self.walk_count + 1 >= 33:
            self.walk_count = 0

        if self.speed > 0:
            screen.blit(self.walk_right[self.walk_count // 3], (self.position_on_x, self.position_on_y))
            self.walk_count += 1
        else:
            screen.blit(self.walk_left[self.walk_count // 3], (self.position_on_x, self.position_on_y))
            self.walk_count += 1

    def move(self):
        if self.speed > 0:
            if self.position_on_x < self.path[1] + self.speed: # if the initial position is less than the end position 
                self.position_on_x += self.speed # walk
            else:
                self.speed *= -1 # go back
                self.position_on_y += self.speed
                self.walk_count = 0
        else:
            if self.position_on_x > self.path[0] - self.speed:
                self.position_on_x += self.speed
            else:
                self.speed *= -1
                self.position_on_x += self.speed
                self.walk_count = 0

Tell draw_game to draw the goblin add to the run_game function an object with Enemy properties  

In [30]:
def draw_game(screen, bullets, goblin):
    screen.blit(Config.bg, (0, 0))
    goblin.draw(screen)
    for bullet in bullets:
        bullet.draw(screen)
    draw_player(screen)
    pg.display.update()


def run_game():
    goblin = Enemy(100, 410, 64, 64, 300)
    bullets = []
    run = True
    screen = pg.display.set_mode((Config.screen_width, Config.screen_height))
    while run:

        Config.clock.tick(27)

        for event in pg.event.get():
            if event.type == pg.QUIT:
                run = False

        for bullet in bullets:
            if 500 > bullet.position_on_x > 0:
                bullet.position_on_x += bullet.speed
            else:
                bullets.pop(bullets.index(bullet))
        keys = pg.key.get_pressed()
        if keys[pg.K_SPACE]:
            if Player.left:
                facing = -1
            else:
                facing = 1

            if len(bullets) < 3:
                bullets.append(
                    Projectile(round(Player.position_on_x + Player.width // 2),
                               round(Player.position_on_y + Player.height // 2), 6, Config.black,
                               facing))
        jump(keys)
        check_move(keys)
        draw_game(screen, bullets, goblin)
    pg.quit()


run_game()